<a href="https://colab.research.google.com/github/Jannat20242NSU/Bangla_NLP_Experiments_BEmoC/blob/main/Model_Implementation_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import sys
import nltk
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score,make_scorer,precision_score, recall_score

#Load Dataset

In [2]:
df_train_wp = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_train_without_preprocess_Final.xlsx')
df_val_wp = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_val_without_preprocess_Final.xlsx')

In [3]:
df_train_cl = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_train_cleaned_Final.xlsx')
df_val_cl = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_val_cleaned_Final.xlsx')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df_train_cl_swr = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_train_cl+swr_Final.xlsx')
df_val_cl_swr = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_val_cl+swr_Final.xlsx')

In [6]:
df_train_cl_lemma = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_train_cl+lemma_Final.xlsx')
df_val_cl_lemma = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_val_cl+lemma_Final.xlsx')

In [7]:
df_train_cl_swr_lemma = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_train_cl+swr+lemma_Final.xlsx')
df_val_cl_swr_lemma = pd.read_excel('/content/drive/MyDrive/UBMEC Datasets/UBMEC_val_cl+swr+lemma_Final.xlsx')

**Drop Null Values**

In [8]:
df_train_wp.dropna(inplace = True)
df_val_wp.dropna(inplace = True)
df_train_cl.dropna(inplace = True)
df_val_cl.dropna(inplace = True)
df_train_cl_swr.dropna(inplace = True)
df_val_cl_swr.dropna(inplace = True)
df_train_cl_lemma.dropna(inplace = True)
df_val_cl_lemma.dropna(inplace = True)
df_train_cl_swr_lemma.dropna(inplace = True)
df_val_cl_swr_lemma.dropna(inplace = True)

#Vectorization Tf-Idf

In [9]:
def create_document_term_matrix_train(sentence_list, vectorizer):
  document_term_matrix = vectorizer.fit_transform(sentence_list)
  return pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [10]:
def create_document_term_matrix_val(sentence_list, vectorizer):
  document_term_matrix = vectorizer.transform(sentence_list)
  return pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [11]:
tfidf = TfidfVectorizer()

In [12]:
tfidf_train_wp = create_document_term_matrix_train(df_train_wp['text'], tfidf)
tfidf_val_wp = create_document_term_matrix_val(df_val_wp['text'], tfidf)

In [13]:
tfidf_train_cl = create_document_term_matrix_train(df_train_cl['text'], tfidf)
tfidf_val_cl = create_document_term_matrix_val(df_val_cl['text'], tfidf)

In [14]:
tfidf_train_cl_swr = create_document_term_matrix_train(df_train_cl_swr['text'], tfidf)
tfidf_val_cl_swr = create_document_term_matrix_val(df_val_cl_swr['text'], tfidf)

In [15]:
tfidf_train_cl_lemma = create_document_term_matrix_train(df_train_cl_lemma['text'], tfidf)
tfidf_val_cl_lemma = create_document_term_matrix_val(df_val_cl_lemma['text'], tfidf)

In [16]:
tfidf_train_cl_swr_lemma = create_document_term_matrix_train(df_train_cl_swr_lemma['text'], tfidf)
tfidf_val_cl_swr_lemma = create_document_term_matrix_val(df_val_cl_swr_lemma['text'], tfidf)

#Model Training

**LabelEncoding**

In [17]:
label_encoder = LabelEncoder()
df_train_wp['encoded_labels'] = label_encoder.fit_transform(df_train_wp['labels'])
df_val_wp['encoded_labels'] = label_encoder.transform(df_val_wp['labels'])


In [18]:
df_train_cl['encoded_labels'] = label_encoder.fit_transform(df_train_cl['labels'])
df_val_cl['encoded_labels'] = label_encoder.transform(df_val_cl['labels'])
df_train_cl_swr['encoded_labels'] = label_encoder.fit_transform(df_train_cl_swr['labels'])
df_val_cl_swr['encoded_labels'] = label_encoder.transform(df_val_cl_swr['labels'])
df_train_cl_lemma['encoded_labels'] = label_encoder.fit_transform(df_train_cl_lemma['labels'])
df_val_cl_lemma['encoded_labels'] = label_encoder.transform(df_val_cl_lemma['labels'])
df_train_cl_swr_lemma['encoded_labels'] = label_encoder.fit_transform(df_train_cl_swr_lemma['labels'])
df_val_cl_swr_lemma['encoded_labels'] = label_encoder.transform(df_val_cl_swr_lemma['labels'])

In [31]:
names = ['Random Forest Tree', 'XGBOOST','Naive Bayes', 'SVM']
classfiers = [
              RandomForestClassifier(),
              xgb.XGBClassifier(),
              MultinomialNB(),
              SVC()

]
models = zip(names, classfiers)

In [32]:
reports = []
def ML_models(X_train,X_test,y_train,y_test):
  for name, model in models:
    model.fit(X_train,y_train)
    test_pred = model.predict(X_test)
    report = classification_report(y_test,test_pred)
    reports.append(report)
  return reports

In [21]:
clf_reports_wp = ML_models(tfidf_train_wp, tfidf_val_wp, df_train_wp['encoded_labels'], df_val_wp['encoded_labels'])

In [24]:
clf_reports_cl = ML_models(tfidf_train_cl, tfidf_val_cl, df_train_cl['encoded_labels'], df_val_cl['encoded_labels'])

In [27]:
clf_reports_cl_swr = ML_models(tfidf_train_cl_swr , tfidf_val_cl_swr , df_train_cl_swr ['encoded_labels'], df_val_cl_swr ['encoded_labels'])

In [30]:
clf_reports_cl_lemma = ML_models(tfidf_train_cl_lemma, tfidf_val_cl_lemma, df_train_cl_lemma['encoded_labels'], df_val_cl_lemma['encoded_labels'])

In [33]:
clf_reports_cl_swr_lemma = ML_models(tfidf_train_cl_swr_lemma, tfidf_val_cl_swr_lemma, df_train_cl_swr_lemma['encoded_labels'], df_val_cl_swr_lemma['encoded_labels'])

In [34]:
for i in range(len(names)):
  print(names[i])
  print("="*100)
  print(clf_reports_wp[i])

Random Forest Tree
              precision    recall  f1-score   support

           0       0.36      0.38      0.37       480
           1       0.31      0.18      0.23       417
           2       0.72      0.45      0.56       277
           3       0.47      0.73      0.57       675
           4       0.39      0.39      0.39       547
           5       0.54      0.24      0.33       268

    accuracy                           0.44      2664
   macro avg       0.46      0.40      0.41      2664
weighted avg       0.44      0.44      0.42      2664

XGBOOST
              precision    recall  f1-score   support

           0       0.34      0.38      0.36       480
           1       0.33      0.23      0.27       417
           2       0.67      0.44      0.53       277
           3       0.48      0.71      0.57       675
           4       0.37      0.35      0.36       547
           5       0.47      0.24      0.32       268

    accuracy                           0.43      2

In [37]:
for i in range(len(names)):
  print(names[i])
  print("="*100)
  print(clf_reports_cl[i])

Random Forest Tree
              precision    recall  f1-score   support

           0       0.37      0.40      0.38       480
           1       0.30      0.16      0.21       417
           2       0.67      0.47      0.55       277
           3       0.47      0.73      0.57       675
           4       0.37      0.39      0.38       547
           5       0.54      0.23      0.32       268

    accuracy                           0.43      2664
   macro avg       0.45      0.39      0.40      2664
weighted avg       0.43      0.43      0.41      2664

XGBOOST
              precision    recall  f1-score   support

           0       0.35      0.39      0.37       480
           1       0.29      0.21      0.25       417
           2       0.64      0.45      0.53       277
           3       0.48      0.71      0.57       675
           4       0.38      0.35      0.36       547
           5       0.46      0.22      0.30       268

    accuracy                           0.42      2

In [38]:
for i in range(len(names)):
  print(names[i])
  print("="*100)
  print(clf_reports_cl_swr[i])

Random Forest Tree
              precision    recall  f1-score   support

           0       0.33      0.41      0.37       480
           1       0.27      0.16      0.20       417
           2       0.66      0.47      0.55       277
           3       0.48      0.67      0.56       670
           4       0.35      0.35      0.35       546
           5       0.45      0.24      0.31       268

    accuracy                           0.41      2658
   macro avg       0.42      0.38      0.39      2658
weighted avg       0.41      0.41      0.40      2658

XGBOOST
              precision    recall  f1-score   support

           0       0.31      0.45      0.37       480
           1       0.27      0.19      0.22       417
           2       0.65      0.45      0.53       277
           3       0.50      0.62      0.55       670
           4       0.36      0.33      0.34       546
           5       0.46      0.24      0.31       268

    accuracy                           0.41      2

In [39]:
for i in range(len(names)):
  print(names[i])
  print("="*100)
  print(clf_reports_cl_lemma[i])

Random Forest Tree
              precision    recall  f1-score   support

           0       0.35      0.38      0.36       480
           1       0.28      0.16      0.20       417
           2       0.67      0.45      0.54       277
           3       0.48      0.72      0.58       675
           4       0.36      0.39      0.37       547
           5       0.54      0.23      0.32       268

    accuracy                           0.43      2664
   macro avg       0.45      0.39      0.40      2664
weighted avg       0.43      0.43      0.41      2664

XGBOOST
              precision    recall  f1-score   support

           0       0.34      0.39      0.36       480
           1       0.31      0.20      0.24       417
           2       0.62      0.46      0.53       277
           3       0.50      0.72      0.59       675
           4       0.36      0.36      0.36       547
           5       0.51      0.22      0.31       268

    accuracy                           0.43      2

In [40]:
for i in range(len(names)):
  print(names[i])
  print("="*100)
  print(clf_reports_cl_swr_lemma[i])

Random Forest Tree
              precision    recall  f1-score   support

           0       0.34      0.34      0.34       480
           1       0.29      0.19      0.23       417
           2       0.64      0.47      0.54       277
           3       0.47      0.70      0.57       670
           4       0.34      0.35      0.34       546
           5       0.46      0.24      0.31       268

    accuracy                           0.41      2658
   macro avg       0.42      0.38      0.39      2658
weighted avg       0.41      0.41      0.40      2658

XGBOOST
              precision    recall  f1-score   support

           0       0.32      0.45      0.37       480
           1       0.28      0.19      0.23       417
           2       0.64      0.44      0.52       277
           3       0.49      0.62      0.55       670
           4       0.36      0.36      0.36       546
           5       0.50      0.21      0.30       268

    accuracy                           0.41      2